<a href="https://colab.research.google.com/github/BaronVonBussin/Stuff/blob/main/NLP_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install -U spacy==3.*
!python -m spacy download en_core_web_sm
!python -m spacy info

2023-08-18 15:10:14.505468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-08-18 15:10:32.469566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.109+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.6.0)        



In [23]:
# Upgrade gensim in case.
# !pip install --upgrade numpy
!pip install -U gensim==4.*

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import random
import spacy

from gensim import models, corpora
from gensim import similarities
from gensim.models.coherencemodel import CoherenceModel
from wordcloud import WordCloud

In [25]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [26]:
!pip install --upgrade --no-cache-dir gdown

In [27]:
# Download the CNN corpus.
!gdown 'https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf'

Downloading...
From (uriginal): https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf
From (redirected): https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf&confirm=t&uuid=729f40a6-b8d5-4432-8822-73b11ea89bc8
To: /content/cnn_articles.txt
100% 365M/365M [00:02<00:00, 132MB/s]


In [29]:
with open('cnn_articles.txt', 'r', encoding='utf8') as f:
  articles = f.read().split('@delimiter')

In [30]:
print(len(articles))
print(articles[0])

92579
 -- Children in war-ravaged Afghanistan are safer than those growing up in London or New York, NATO's top civilian envoy says.

Mark Sedwill, the senior civilian representative for NATO, made the comments on an episode of CBBC's "Newsround," which is airing Monday.

In the show -- a BBC current-affairs program for children -- several youngsters in Kabul, Afghanistan, say they are afraid of daily violence and the frequent explosions in their war-torn country.

In response, Sedwill says: "Here in Kabul, and other big cities actually, there are very few of these bombs. The children are probably safer here than they would be in London, New York or Glasgow or many other cities.

"Most children can go about their lives in safety. It's a very family-oriented society. So it is a little bit like a city of villages," he added.

A U.N. report released earlier this year seems to contradict Sedwill's assessment.

The February report, by the special representative for children and armed confli

In [31]:
DATASET_SIZE = 20000
dataset = articles[:DATASET_SIZE]

In [32]:
nlp = spacy.blank('en')

def basic_filter(tokenized_doc):
  return [t.text for t in tokenized_doc if
          not t.is_punct and \
          not t.is_space and \
          t.is_alpha]

In [33]:
NUM_PROCESS = 4

In [34]:
%%time
tokenized_articles = list(map(basic_filter, nlp.pipe(dataset, n_process=NUM_PROCESS)))

CPU times: user 2min 22s, sys: 2.58 s, total: 2min 24s
Wall time: 3min 10s


In [35]:
print(tokenized_articles[0])

['Children', 'in', 'war', 'ravaged', 'Afghanistan', 'are', 'safer', 'than', 'those', 'growing', 'up', 'in', 'London', 'or', 'New', 'York', 'NATO', 'top', 'civilian', 'envoy', 'says', 'Mark', 'Sedwill', 'the', 'senior', 'civilian', 'representative', 'for', 'NATO', 'made', 'the', 'comments', 'on', 'an', 'episode', 'of', 'CBBC', 'Newsround', 'which', 'is', 'airing', 'Monday', 'In', 'the', 'show', 'a', 'BBC', 'current', 'affairs', 'program', 'for', 'children', 'several', 'youngsters', 'in', 'Kabul', 'Afghanistan', 'say', 'they', 'are', 'afraid', 'of', 'daily', 'violence', 'and', 'the', 'frequent', 'explosions', 'in', 'their', 'war', 'torn', 'country', 'In', 'response', 'Sedwill', 'says', 'Here', 'in', 'Kabul', 'and', 'other', 'big', 'cities', 'actually', 'there', 'are', 'very', 'few', 'of', 'these', 'bombs', 'The', 'children', 'are', 'probably', 'safer', 'here', 'than', 'they', 'would', 'be', 'in', 'London', 'New', 'York', 'or', 'Glasgow', 'or', 'many', 'other', 'cities', 'Most', 'children

In [36]:
NUM_TOPICS = 20

In [37]:
# Build a Dictionary of word<-->id mappings.
%%time
dictionary = corpora.Dictionary(tokenized_articles)

sample_token = 'news'
print(f'Id for \'{sample_token}\' token: {dictionary.token2id[sample_token]}')

Id for 'news' token: 1039
CPU times: user 12.2 s, sys: 26 ms, total: 12.2 s
Wall time: 12.3 s


In [38]:
%%time
corpus_bow = [dictionary.doc2bow(article) for article in tokenized_articles]

CPU times: user 7.14 s, sys: 23.9 ms, total: 7.17 s
Wall time: 7.2 s


In [39]:
%%time
lda_model = models.LdaModel(corpus=corpus_bow, num_topics=NUM_TOPICS, id2word=dictionary, random_state=1)

CPU times: user 54.6 s, sys: 38.9 s, total: 1min 33s
Wall time: 55 s


In [41]:
lda_model.print_topics()

[(0,
  '0.068*"the" + 0.041*"of" + 0.027*"and" + 0.025*"to" + 0.025*"in" + 0.023*"a" + 0.013*"is" + 0.009*"for" + 0.008*"that" + 0.008*"The"'),
 (1,
  '0.046*"the" + 0.031*"I" + 0.030*"a" + 0.028*"to" + 0.022*"and" + 0.019*"in" + 0.017*"of" + 0.012*"was" + 0.012*"it" + 0.012*"that"'),
 (2,
  '0.064*"the" + 0.030*"of" + 0.025*"to" + 0.023*"and" + 0.020*"in" + 0.017*"a" + 0.011*"that" + 0.011*"is" + 0.009*"Syria" + 0.008*"for"'),
 (3,
  '0.038*"the" + 0.032*"to" + 0.029*"and" + 0.029*"of" + 0.024*"a" + 0.017*"that" + 0.016*"in" + 0.013*"is" + 0.010*"are" + 0.009*"for"'),
 (4,
  '0.069*"the" + 0.030*"of" + 0.029*"to" + 0.024*"and" + 0.021*"in" + 0.019*"a" + 0.015*"said" + 0.011*"The" + 0.010*"that" + 0.009*"on"'),
 (5,
  '0.060*"the" + 0.031*"in" + 0.029*"of" + 0.028*"to" + 0.027*"and" + 0.021*"a" + 0.014*"said" + 0.011*"that" + 0.011*"The" + 0.008*"is"'),
 (6,
  '0.054*"the" + 0.033*"to" + 0.030*"of" + 0.027*"in" + 0.026*"and" + 0.025*"a" + 0.012*"said" + 0.011*"that" + 0.009*"The" + 0.0

In [42]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def improved_filter(tokenized_doc):
  return [t.lemma_ for t in tokenized_doc if
          t.is_alpha and \
          not t.is_punct and \
          not t.is_space and \
          not t.is_stop and \
          t.pos_ in ['NOUN', 'VERB', 'ADJ']]

In [43]:
# We'll need to retokenize everything and rebuild the BOWs. Because we're now
# using the POS tagger, this will take longer. The "w_pos" in the variable
# names below just means "with part-of-speech".
%%time
tokenized_articles_w_pos = list(map(improved_filter, nlp.pipe(dataset, n_process=NUM_PROCESS)))
dictionary_w_pos = corpora.Dictionary(tokenized_articles_w_pos)
corpus_bow_w_pos = [dictionary_w_pos.doc2bow(article) for article in tokenized_articles_w_pos]

KeyboardInterrupt: ignored

In [21]:
%%time
lda_model = models.LdaModel(corpus=corpus_bow_w_pos, num_topics=NUM_TOPICS, id2word=dictionary_w_pos, random_state=1)

NameError: ignored

In [ ]:
lda_model.print_topics()

In [ ]:
# The size of the dictionary before filtering.
len(dictionary_w_pos)

In [ ]:
dictionary_w_pos.filter_extremes(no_below=5, no_above=0.5)

In [ ]:
# The size of the dictionary after filtering.
len(dictionary_w_pos)

In [ ]:
# Rebuild bag of words.
corpus_bow_w_pos_filtered = [dictionary_w_pos.doc2bow(article) for article in tokenized_articles_w_pos]

In [ ]:
%%time
lda_model = models.ldamodel.LdaModel(corpus=corpus_bow_w_pos_filtered,
                                     id2word=dictionary_w_pos,
                                     num_topics=NUM_TOPICS,
                                     passes=10,
                                     alpha='auto',
                                     eta='auto',
                                     random_state=1)

In [ ]:
lda_model.print_topics()

In [ ]:
print(lda_model.alpha)
print(lda_model.eta)

In [ ]:
article_idx = 0
print(dataset[article_idx][:300])

In [ ]:
# Return topic distribution for an article sorted by probability.
topics = sorted(lda_model.get_document_topics(corpus_bow_w_pos_filtered[article_idx]), key=lambda tup: tup[1])[::-1]
topics

In [ ]:
# View the words of the top topic from the previous article.
lda_model.show_topic(topics[0][0])

In [ ]:
# View the words of the second-most prevalent topic from the previous article.
lda_model.show_topic(topics[1][0])

In [ ]:
def get_top_topics(article_idx, min_topic_prob):

  # Sort from highest to lowest topic probability.
  topic_prob_pairs = sorted(lda_model.get_document_topics(corpus_bow_w_pos_filtered[article_idx],
                                                          minimum_probability=min_topic_prob),
                            key=lambda tup: tup[1])[::-1]

  word_prob_pairs = [lda_model.show_topic(pair[0]) for pair in topic_prob_pairs]
  topic_words = [[pair[0] for pair in collection] for collection in word_prob_pairs]

  data = {
      'Major Topics': topic_prob_pairs,
      'Topic Words': topic_words
  }

  return pd.DataFrame(data)

In [ ]:
pd.set_option('max_colwidth', 600)
snippet_length = 300
min_topic_prob = 0.25

article_idx = 1
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)

In [ ]:
article_idx = 10
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)

In [ ]:
article_idx = 100
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)

In [ ]:
article_idx = 1000
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)

In [ ]:
article_idx = 10000
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, 0.25)

In [ ]:
%%time
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_articles_w_pos, dictionary=dictionary_w_pos, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
def render_word_cloud(model, rows, cols, max_words):
  word_cloud = WordCloud(background_color='white', max_words=max_words, prefer_horizontal=1.0)
  fig, axes = plt.subplots(rows, cols, figsize=(15,15))

  for i, ax in enumerate(axes.flatten()):
      fig.add_subplot(ax)
      topic_words = dict(model.show_topic(i))
      word_cloud.generate_from_frequencies(topic_words)
      plt.gca().imshow(word_cloud, interpolation='bilinear')
      plt.gca().set_title('Topic {id}'.format(id=i))
      plt.gca().axis('off')

  plt.axis('off')
  plt.show()

In [ ]:
# Here we'll visualize the first nine topics.
render_word_cloud(lda_model, 3, 3, 10)

In [ ]:
lda_index = similarities.MatrixSimilarity(lda_model[corpus_bow_w_pos_filtered], num_features=len(dictionary_w_pos))

In [ ]:
def get_similar_articles(index, model, article_bow, top_n=5, first_m_words=300):
  # model[article_bow] retrieves the topic distribution for the BOW.
  # index[model[article_bow] compares the topic distribution for the BOW against the similarity index previously computed.
  similar_docs = index[model[article_bow]]
  top_n_docs = sorted(enumerate(similar_docs), key=lambda item: -item[1])[1:top_n+1]

  # Return a list of tuples with each tuple: (article id, similarity score, first_m_words of article)
  return list(map(lambda entry: (entry[0], entry[1], articles[entry[0]][:first_m_words]), top_n_docs))

In [ ]:
article_idx = 0
print(dataset[article_idx][:snippet_length], '\n')
get_similar_articles(lda_index, lda_model, corpus_bow_w_pos_filtered[article_idx])

In [ ]:
article_idx = 10
print(dataset[article_idx][:snippet_length], '\n')
get_similar_articles(lda_index, lda_model, corpus_bow_w_pos_filtered[article_idx])

In [ ]:
article_idx = 100
print(dataset[article_idx][:snippet_length], '\n')
get_similar_articles(lda_index, lda_model, corpus_bow_w_pos_filtered[article_idx])

In [ ]:
test_article = "Capricorn Business Acquisitions Inc. (TSXV: CAK.H) (the “Company“) is pleased to announce that its board has approved the issuance of 70,000 stock options (“Stock Options“) to directors on April 19, 2020."

article_tokens = list(map(improved_filter, [nlp(test_article)]))[0]
article_bow = dictionary_w_pos.doc2bow(article_tokens)
get_similar_articles(lda_index, lda_model, article_bow)

In [ ]:
test_article = "DEA agent sentenced to 12 years in prison for conspiring with Colombian drug cartel."

article_tokens = list(map(improved_filter, [nlp(test_article)]))[0]
article_bow = dictionary_w_pos.doc2bow(article_tokens)
get_similar_articles(lda_index, lda_model, article_bow)